In [1]:
import yggdrasilDecisionForests.yggdrasil_decision_forests as ydf


treeple1 environment

In [2]:
! pip install ydf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 41.5 MB/s eta 0:00:00a 0:00:01


In [1]:
import ydf

In [4]:
! pip install pandas

  Using cached pandas-2.2.3-cp310-cp310-macosx_11_0_arm64.whl.metadata (89 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.2.3-cp310-cp310-macosx_11_0_arm64.whl (11.3 MB)
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)


In [2]:
import treeple
from treeple.datasets import make_trunk_classification
import pandas as pd
import time
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [3]:
def prepare_dataframe(X, y):
    df = pd.DataFrame(X)
    df.columns = [str(i) for i in df.columns]  # Convert column names to strings
    df["target"] = y.astype(int)  # Append target column

    return df

def train_ydf(X_train, y_train, X_test, y_test, params_ydf, n_dim=500):
    df_train = prepare_dataframe(X_train, y_train)
    df_test = prepare_dataframe(X_test, y_test)

    learner = ydf.RandomForestLearner(**params_ydf)
    start_time = time.time()
    ydf_model = learner.train(df_train)
    time_ydf = time.time() - start_time
    y_pred = ydf_model.predict(df_test)
    y_pred_class = (y_pred >= 0.5).astype(int) 

    acc_ydf = accuracy_score(y_test, y_pred_class)

    print(f"YDF | n_dim: {n_dim} | n_tree: {params_ydf['num_trees']} | Accuracy: {acc_ydf:.4f} | Train Time: {time_ydf:.4f} sec")
    return acc_ydf, time_ydf, y_pred_class

In [4]:
MAX_DEPTH = 10
N_ESTIMATORS = 300
RANDOM_SEED = 42
N_JOBS=-1
BOOTSTRAP = True
MAX_FEATURE = 100
FEATURE_COMBINATIONS = 1.0

params_ydf = dict()
params_ydf["label"] = "target"
params_ydf["max_depth"] = MAX_DEPTH
params_ydf["num_trees"] = N_ESTIMATORS
params_ydf["random_seed"] = RANDOM_SEED
params_ydf["honest"] = False
params_ydf["bootstrap_size_ratio"] = 1.0
params_ydf["bootstrap_training_dataset"] = BOOTSTRAP
params_ydf["categorical_algorithm"] = "ONE_HOT"
params_ydf["compute_oob_performances"] = False  #oob_score
params_ydf["compute_oob_variable_importances"] = False  ## oob_score
params_ydf["in_split_min_examples_check"] = True
params_ydf["min_examples"] = 1    #min_samples_leaf
params_ydf["keep_non_leaf_label_distribution"] = True   # not sure
params_ydf["max_num_nodes"] = None
params_ydf["growing_strategy"] = "BEST_FIRST_GLOBAL"
# params_ydf["num_candidate_attributes"] = -1  #0 equivalent to sqrt max_features in treeple
params_ydf["num_candidate_attributes_ratio"] = 1.0
params_ydf["sorting_strategy"] = "IN_NODE" ###
# sparse oblique params
params_ydf["split_axis"] = "SPARSE_OBLIQUE"
params_ydf["sparse_oblique_weights"] = "CONTINUOUS"
params_ydf["sparse_oblique_projection_density_factor"] = FEATURE_COMBINATIONS
params_ydf["sparse_oblique_normalization"] = "MIN_MAX"
params_ydf["sparse_oblique_max_num_projections"] = MAX_FEATURE # modify this, find in sourse code in treeple
params_ydf["sparse_oblique_num_projections_exponent"] = 1.0   #2.0
params_ydf["sampling_with_replacement"] = False

In [36]:
n_samples=2000
num_dim=10
X, y = make_trunk_classification(n_samples=n_samples, n_dim=num_dim, n_informative=600, seed=0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

df_train = prepare_dataframe(X_train, y_train)
df_test = prepare_dataframe(X_test, y_test)

c:\Users\clark\anaconda3\envs\catboost_env\lib\site-packages\treeple\datasets\hyppo.py:515: RuntimeWarning: Number of informative dimensions {n_informative} must be less than number of dimensions, 10. Setting n_informative to n_dim.
  warnings.warn(


In [37]:
learner = ydf.RandomForestLearner(**params_ydf)
start_time = time.time()
ydf_model = learner.train(df_train)
time_ydf = time.time() - start_time
pred_ydf = ydf_model.predict(df_test)
pred_ydf = (pred_ydf >= 0.5).astype(int) 

acc_ydf = accuracy_score(y_test, pred_ydf)
acc_ydf

Train model on 1600 examples
Model trained in 0:00:00.122748


0.74

In [38]:
learner.hyperparameter_templates()

{'better_defaultv1': HyperparameterTemplate(name='better_default', version=1, parameters={'winner_take_all': True}, description='A configuration that is generally better than the default parameters without being more expensive.'),
 'benchmark_rank1v1': HyperparameterTemplate(name='benchmark_rank1', version=1, parameters={'winner_take_all': True, 'categorical_algorithm': 'RANDOM', 'split_axis': 'SPARSE_OBLIQUE', 'sparse_oblique_normalization': 'MIN_MAX', 'sparse_oblique_num_projections_exponent': 1.0}, description='Top ranking hyper-parameters on our benchmark slightly modified to run in reasonable time.')}

In [39]:
ydf_model.describe()

In [9]:
ydf_model.save("saved_model/")

In [40]:
ydf_model.data_spec()

columns {
  type: CATEGORICAL
  name: "target"
  categorical {
    number_of_unique_values: 3
    items {
      key: "<OOD>"
      value {
        index: 0
        count: 0
      }
    }
    items {
      key: "1"
      value {
        index: 2
        count: 800
      }
    }
    items {
      key: "0"
      value {
        index: 1
        count: 800
      }
    }
  }
  count_nas: 0
  dtype: DTYPE_INT32
}
columns {
  type: NUMERICAL
  name: "0"
  numerical {
    mean: 0.50206137866509382
    min_value: -3.38852024
    max_value: 3.77264
    standard_deviation: 1.0926331375419041
  }
  count_nas: 0
  dtype: DTYPE_FLOAT64
}
columns {
  type: NUMERICAL
  name: "1"
  numerical {
    mean: 0.32230514054055676
    min_value: -3.55543637
    max_value: 4.28789711
    standard_deviation: 1.051411575474217
  }
  count_nas: 0
  dtype: DTYPE_FLOAT64
}
columns {
  type: NUMERICAL
  name: "2"
  numerical {
    mean: 0.26846194128011119
    min_value: -3.21311045
    max_value: 3.50855422
    stan

In [41]:
ydf_tree = ydf_model.get_tree(1)
ydf_tree

Tree(root=NonLeaf(value=ProbabilityValue(num_examples=1600.0, probability=[0.5, 0.5]), condition=NumericalSparseObliqueCondition(missing=False, score=0.053251296281814575, attributes=[2, 5, 6], weights=[-0.052857689559459686, -0.12740656733512878, -0.0796140506863594], threshold=-0.044604748487472534), pos_child=NonLeaf(value=ProbabilityValue(num_examples=742.0, probability=[0.6738544474393531, 0.3261455525606469]), condition=NumericalSparseObliqueCondition(missing=False, score=0.040539998561143875, attributes=[1, 3], weights=[-0.0028991461731493473, -0.08910401165485382], threshold=-0.008685402572154999), pos_child=NonLeaf(value=ProbabilityValue(num_examples=342.0, probability=[0.8157894736842105, 0.18421052631578946]), condition=NumericalSparseObliqueCondition(missing=False, score=0.04142618179321289, attributes=[4], weights=[1.0], threshold=0.04757502302527428), pos_child=NonLeaf(value=ProbabilityValue(num_examples=183.0, probability=[0.7158469945355191, 0.28415300546448086]), condi

In [43]:
ydf_root = ydf_tree.root
ydf_root

NonLeaf(value=ProbabilityValue(num_examples=1600.0, probability=[0.5, 0.5]), condition=NumericalSparseObliqueCondition(missing=False, score=0.053251296281814575, attributes=[2, 5, 6], weights=[-0.052857689559459686, -0.12740656733512878, -0.0796140506863594], threshold=-0.044604748487472534), pos_child=NonLeaf(value=ProbabilityValue(num_examples=742.0, probability=[0.6738544474393531, 0.3261455525606469]), condition=NumericalSparseObliqueCondition(missing=False, score=0.040539998561143875, attributes=[1, 3], weights=[-0.0028991461731493473, -0.08910401165485382], threshold=-0.008685402572154999), pos_child=NonLeaf(value=ProbabilityValue(num_examples=342.0, probability=[0.8157894736842105, 0.18421052631578946]), condition=NumericalSparseObliqueCondition(missing=False, score=0.04142618179321289, attributes=[4], weights=[1.0], threshold=0.04757502302527428), pos_child=NonLeaf(value=ProbabilityValue(num_examples=183.0, probability=[0.7158469945355191, 0.28415300546448086]), condition=Numer

In [44]:
ydf_root.neg_child

NonLeaf(value=ProbabilityValue(num_examples=858.0, probability=[0.34965034965034963, 0.6503496503496503]), condition=NumericalSparseObliqueCondition(missing=False, score=0.027874784544110298, attributes=[2], weights=[1.0], threshold=0.8453803062438965), pos_child=NonLeaf(value=ProbabilityValue(num_examples=354.0, probability=[0.2175141242937853, 0.7824858757062146]), condition=NumericalSparseObliqueCondition(missing=False, score=0.02652428299188614, attributes=[3], weights=[1.0], threshold=0.36735761165618896), pos_child=NonLeaf(value=ProbabilityValue(num_examples=187.0, probability=[0.12834224598930483, 0.8716577540106952]), condition=NumericalSparseObliqueCondition(missing=False, score=0.034788746386766434, attributes=[8], weights=[1.0], threshold=0.052107613533735275), pos_child=Leaf(value=ProbabilityValue(num_examples=110.0, probability=[0.05454545454545454, 0.9454545454545454])), neg_child=NonLeaf(value=ProbabilityValue(num_examples=77.0, probability=[0.23376623376623376, 0.766233

In [45]:
root_str = str(ydf_root)

In [71]:
import re
import numpy as np
from scipy.sparse import lil_matrix

def extract_projections(tree_str):
    
    attr_blocks = re.findall(r'attributes=\[([0-9,\s]+)\]\s*,\s*weights=\[([0-9eE+.\-,\s]+)\]', tree_str)

    projection_rows = []
    max_feature_index = -1

    for attr_str, weight_str in attr_blocks:
        attrs = [int(i.strip()) for i in attr_str.split(',') if i.strip()]
        weights = [float(w.strip()) for w in weight_str.split(',') if w.strip()]
        
        if len(attrs) != len(weights):
            raise ValueError(f"Mismatched attributes and weights: {attrs}, {weights}")
        
        # Track the maximum feature index
        if attrs:
            max_feature_index = max(max_feature_index, max(attrs))
        
        projection_rows.append((attrs, weights))

    # Projection dimension = max feature index + 1
    proj_dim = max_feature_index + 1

    # Initialize sparse projection matrix
    projection_matrix = lil_matrix((len(projection_rows), proj_dim))
    for i, (attrs, weights) in enumerate(projection_rows):
        for j, attr in enumerate(attrs):
            projection_matrix[i, attr] = weights[j]

    return projection_matrix, proj_dim


In [46]:
projection_matrix, proj_dim = extract_projections(root_str)
projection_matrix.shape

(30, 11)

In [47]:
projection_matrix2, proj_dim2 = extract_projections(str(ydf_root.neg_child))
projection_matrix2.shape

(17, 11)

In [48]:
projection_matrix3, proj_dim3 = extract_projections(str(ydf_root.pos_child))
projection_matrix3.shape

(12, 10)

In [49]:
proj_dim
projection_matrix.shape

(30, 11)

In [42]:
len(ydf_model.get_all_trees())

300

Do a pipline

In [56]:
params_ydf

{'label': 'target',
 'max_depth': 10,
 'num_trees': 300,
 'random_seed': 42,
 'honest': False,
 'bootstrap_size_ratio': 1.0,
 'bootstrap_training_dataset': True,
 'categorical_algorithm': 'ONE_HOT',
 'compute_oob_performances': False,
 'compute_oob_variable_importances': False,
 'in_split_min_examples_check': True,
 'min_examples': 1,
 'keep_non_leaf_label_distribution': True,
 'max_num_nodes': None,
 'growing_strategy': 'BEST_FIRST_GLOBAL',
 'num_candidate_attributes_ratio': 1.0,
 'sorting_strategy': 'IN_NODE',
 'split_axis': 'SPARSE_OBLIQUE',
 'sparse_oblique_weights': 'CONTINUOUS',
 'sparse_oblique_projection_density_factor': 1.0,
 'sparse_oblique_normalization': 'MIN_MAX',
 'sparse_oblique_max_num_projections': 100,
 'sparse_oblique_num_projections_exponent': 1.0,
 'sampling_with_replacement': False}

In [ ]:
def extract_projections_pipline(params_ydf, n_sample=2000, n_estimator=100, n_dim=50, max_feature=100, feature_combination=3.0, exponent=1.0):
    
    X, y = make_trunk_classification(n_samples=n_sample, n_dim=n_dim, n_informative=600, seed=0)
    X_train, _, y_train, _ = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    df_train = prepare_dataframe(X_train, y_train)

    params_ydf1 = params_ydf.copy()
    params_ydf1["num_trees"] = n_estimator
    params_ydf1["sparse_oblique_max_num_projections"] = max_feature
    params_ydf1["sparse_oblique_projection_density_factor"] = feature_combination
    params_ydf1["sparse_oblique_num_projections_exponent"] = exponent


    learner = ydf.RandomForestLearner(**params_ydf1)
    start_time = time.time()
    ydf_model = learner.train(df_train)
    time_taken = time.time() - start_time

    root_str = str(ydf_model.get_tree(0).root)
    print("*",root_str)
    projection_matrix, proj_dim = extract_projections(root_str)

    return projection_matrix, proj_dim, time_taken, ydf_model

In [ ]:
projection_matrix, proj_dim, _, _ = extract_projections_pipline(params_ydf,
                                                n_sample=2000, 
                                                n_estimator=100, 
                                                n_dim=50, 
                                                max_feature=100, 
                                                feature_combination=3.0)
projection_matrix.shape

Train model on 1600 examples


c:\Users\clark\anaconda3\envs\catboost_env\lib\site-packages\treeple\datasets\hyppo.py:515: RuntimeWarning: Number of informative dimensions {n_informative} must be less than number of dimensions, 50. Setting n_informative to n_dim.
  warnings.warn(


Model trained in 0:00:00.233449
* NonLeaf(value=ProbabilityValue(num_examples=1600.0, probability=[0.5, 0.5]), condition=NumericalSparseObliqueCondition(missing=False, score=0.10298066586256027, attributes=[1], weights=[1.0], threshold=0.40478795766830444), pos_child=NonLeaf(value=ProbabilityValue(num_examples=846.0, probability=[0.2895981087470449, 0.710401891252955]), condition=NumericalSparseObliqueCondition(missing=False, score=0.050919171422719955, attributes=[2], weights=[1.0], threshold=0.702617883682251), pos_child=NonLeaf(value=ProbabilityValue(num_examples=361.0, probability=[0.12742382271468145, 0.8725761772853186]), condition=NumericalSparseObliqueCondition(missing=False, score=0.03299005702137947, attributes=[1], weights=[1.0], threshold=1.014682650566101), pos_child=NonLeaf(value=ProbabilityValue(num_examples=229.0, probability=[0.0611353711790393, 0.9388646288209607]), condition=NumericalSparseObliqueCondition(missing=False, score=0.04389900341629982, attributes=[3, 12, 

(30, 50)

In [ ]:
projection_matrix, proj_dim, _, _ = extract_projections_pipline(params_ydf,
                                                n_sample=2000, 
                                                n_estimator=100, 
                                                n_dim=200, 
                                                max_feature=100, 
                                                feature_combination=3.0)
projection_matrix.shape

Train model on 1600 examples


c:\Users\clark\anaconda3\envs\catboost_env\lib\site-packages\treeple\datasets\hyppo.py:515: RuntimeWarning: Number of informative dimensions {n_informative} must be less than number of dimensions, 200. Setting n_informative to n_dim.
  warnings.warn(


Model trained in 0:00:00.524940
* NonLeaf(value=ProbabilityValue(num_examples=1600.0, probability=[0.5, 0.5]), condition=NumericalSparseObliqueCondition(missing=False, score=0.09011893719434738, attributes=[1], weights=[1.0], threshold=0.39532381296157837), pos_child=NonLeaf(value=ProbabilityValue(num_examples=838.0, probability=[0.30071599045346065, 0.6992840095465394]), condition=NumericalSparseObliqueCondition(missing=False, score=0.018235361203551292, attributes=[2, 107, 127, 167], weights=[0.14494670927524567, 0.13897280395030975, 0.07349523156881332, 0.0880846455693245], threshold=0.1965847909450531), pos_child=NonLeaf(value=ProbabilityValue(num_examples=278.0, probability=[0.17985611510791366, 0.8201438848920863]), condition=NumericalSparseObliqueCondition(missing=False, score=0.03449970856308937, attributes=[37, 42, 111, 146, 193], weights=[0.008129679597914219, 0.09601710736751556, 0.14890573918819427, 0.11647744476795197, 0.047354381531476974], threshold=0.2536314129829407), 

(30, 198)

In [ ]:
projection_matrix, proj_dim, _, _ = extract_projections_pipline(params_ydf,
                                                n_sample=2000, 
                                                n_estimator=100, 
                                                n_dim=1000, 
                                                max_feature=300, 
                                                feature_combination=1.0)
projection_matrix.shape

Train model on 1600 examples
Model trained in 0:00:02.524466
* NonLeaf(value=ProbabilityValue(num_examples=1600.0, probability=[0.5, 0.5]), condition=NumericalSparseObliqueCondition(missing=False, score=0.02640022709965706, attributes=[4], weights=[1.0], threshold=-0.0243707075715065), pos_child=NonLeaf(value=ProbabilityValue(num_examples=942.0, probability=[0.40445859872611467, 0.5955414012738853]), condition=NumericalSparseObliqueCondition(missing=False, score=0.04424554854631424, attributes=[2], weights=[1.0], threshold=1.0494449138641357), pos_child=NonLeaf(value=ProbabilityValue(num_examples=260.0, probability=[0.17692307692307693, 0.823076923076923]), condition=NumericalSparseObliqueCondition(missing=False, score=0.07184826582670212, attributes=[1], weights=[1.0], threshold=0.3606805205345154), pos_child=Leaf(value=ProbabilityValue(num_examples=176.0, probability=[0.07386363636363637, 0.9261363636363636])), neg_child=NonLeaf(value=ProbabilityValue(num_examples=84.0, probability=[

(30, 956)

In [ ]:
projection_matrix, proj_dim, _, _ = extract_projections_pipline(params_ydf,
                                                n_sample=2000, 
                                                n_estimator=100, 
                                                n_dim=1000, 
                                                max_feature=300, 
                                                feature_combination=1.0,
                                                exponent=100.0)
projection_matrix.shape

Train model on 1600 examples
Model trained in 0:00:00.038572
* NonLeaf(value=ProbabilityValue(num_examples=1600.0, probability=[0.5, 0.5]), condition=NumericalSparseObliqueCondition(missing=False, score=0.0023543008137494326, attributes=[513], weights=[1.0], threshold=-0.20640619099140167), pos_child=NonLeaf(value=ProbabilityValue(num_examples=942.0, probability=[0.4713375796178344, 0.5286624203821656]), condition=NumericalSparseObliqueCondition(missing=False, score=0.01493818685412407, attributes=[6], weights=[1.0], threshold=0.8464932441711426), pos_child=Leaf(value=ProbabilityValue(num_examples=247.0, probability=[0.32793522267206476, 0.6720647773279352])), neg_child=NonLeaf(value=ProbabilityValue(num_examples=695.0, probability=[0.5223021582733813, 0.4776978417266187]), condition=NumericalSparseObliqueCondition(missing=False, score=0.0037507221568375826, attributes=[236, 246], weights=[-0.12089046090841293, 0.0034681425895541906], threshold=-0.2745153605937958), pos_child=NonLeaf(v

(30, 1001)